In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import datasets
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import os
import json
from tqdm import tqdm
from datetime import datetime
from omegaconf import OmegaConf
import argparse

/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-19 09:26:29.787668: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 09:26:29.813511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 09:26:30.127797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [2]:
def make_dataset(train_data_path, validation_data_path):
    loaded_data_dict = {
        'train': {
            'err_sentence': [],
            'cor_sentence': [],
        },
        'validation': {
            'err_sentence': [],
            'cor_sentence': [],
        }
    }

    # Train 데이터 로드
    with open(train_data_path, 'r') as f:
        _temp_json = json.load(f)
    loaded_data_dict['train']['err_sentence'].extend(list(map(lambda x: x['annotation']['err_sentence'], _temp_json['data'])))
    loaded_data_dict['train']['cor_sentence'].extend(list(map(lambda x: x['annotation']['cor_sentence'], _temp_json['data'])))
    print(f'Train data loaded: {len(_temp_json["data"])} entries')

    # Validation 데이터 로드
    with open(validation_data_path, 'r') as f:
        _temp_json = json.load(f)
    loaded_data_dict['validation']['err_sentence'].extend(list(map(lambda x: x['annotation']['err_sentence'], _temp_json['data'])))
    loaded_data_dict['validation']['cor_sentence'].extend(list(map(lambda x: x['annotation']['cor_sentence'], _temp_json['data'])))
    print(f'Validation data loaded: {len(_temp_json["data"])} entries')

    # DatasetDict 생성
    dataset_dict = {}
    for _trg in loaded_data_dict.keys():
        dataset_dict[_trg] = datasets.Dataset.from_dict(loaded_data_dict[_trg])
    dataset = datasets.DatasetDict(dataset_dict)
    return dataset


In [3]:
def preprocess_function(df, tokenizer, src_col, tgt_col, max_length):
    '''
    This is a function for preprocessing dataset.
    
    Args:
        df (Dataset): A data in the dataset.
        tokenizer (AutoTokenizer): Model tokenizer.
        src_col (str): Source column name.
        tgt_col (str): Target column name.
        max_length (int): Max length.
    Returns:
        model_inputs (Dataset):
            - input_ids (list): Input data.
            - labels (list): Labeled Data.
    '''
    inputs = df[src_col]
    targets = df[tgt_col]
    model_inputs = tokenizer(
        inputs,
        max_length=max_length,
        truncation=True,
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True)
        
    model_inputs["labels"] = labels['input_ids']
    return model_inputs


In [4]:
def train(config):
    '''
    This is a function for training.
    
    Args:
        config (Dict): Config dict is made by config.yaml
    '''
    # Load model and tokenizer
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Model Load Start ======')
    model = AutoModelForSeq2SeqLM.from_pretrained(config.pretrained_model_name)
    tokenizer = AutoTokenizer.from_pretrained(config.pretrained_model_name)
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Model Load Finished ======')
    
    # Load data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    
    # load data and set data
    print(f'[{_now_time}] ====== Data Load Start ======')
    _now_time = datetime.now().__str__()
    dataset = make_dataset('/home/yjtech2/Desktop/yurim/LLM/Data/train/맞춤법오류_자유게시판.json', '/home/yjtech2/Desktop/yurim/LLM/Data/validation/맞춤법오류_자유게시판.json')
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Finished ======')
    
    # data preprocessing
    print(f'[{_now_time}] ====== Data Preprocessing Start ======')
    _now_time = datetime.now().__str__()
    dataset_tokenized = dataset.map(lambda d: preprocess_function(d, tokenizer, config.src_col, config.tgt_col, config.max_length), batched=True, batch_size=config.per_device_train_batch_size)
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Preprocessing Finished ======')
    
    # set training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.output_dir,
        learning_rate=config.learning_rate,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=config.per_device_eval_batch_size,
        num_train_epochs=config.num_train_epochs,
        fp16=config.fp16,
        weight_decay=config.weight_decay,
        do_eval=config.do_eval,
        evaluation_strategy=config.evaluation_strategy,
        warmup_ratio=config.warmup_ratio,
        log_level=config.log_level,
        logging_dir=config.logging_dir,
        logging_strategy=config.logging_strategy,
        logging_steps=config.logging_steps,
        eval_steps=config.eval_steps,
        save_strategy=config.save_strategy,
        save_steps=config.save_steps,
        save_total_limit=config.save_total_limit,
        load_best_model_at_end=config.load_best_model_at_end,
        metric_for_best_model=config.metric_for_best_model,
        greater_is_better=config.greater_is_better,
        dataloader_num_workers=config.dataloader_num_workers,
        group_by_length=config.group_by_length,
        report_to=config.report_to,
        ddp_find_unused_parameters=config.ddp_find_unused_parameters,
     )
    
    # set trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset_tokenized['train'],
        eval_dataset=dataset_tokenized['validation'],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    # start train
    trainer.train()

In [5]:
if __name__ == '__main__':
    '''
    Usage:
        python train.py --config_file config/base-config.yaml
    '''
    # parse inputs args
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--config_file')
    # args = parser.parse_args(sys.argv[1:])
    
    # load config file
    # config_file = args.config_file
    config_file = '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/config/base-config.yaml'
    config = OmegaConf.load(config_file)
    
    # make save path
    save_path = './data/results'
    os.makedirs(save_path, exist_ok=True)
    
    # set device
    os.environ["CUDA_VISIBLE_DEVICES"] = config.CUDA_VISIBLE_DEVICES
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ========== Train Start ==========')
    
    # call main method
    print(f'DEVICE : {config.CUDA_VISIBLE_DEVICES}')
    print(f'MODEL NAME : {config.pretrained_model_name}')
    # print(f'TRAIN FILE PATH :')
    # for _path in config.train_data_path_list:
    #     print(f' - {_path}')
    # print(f'VALIDATION FILE PATH :')
    # for _path in config.validation_data_path_list:
    #     print(f' - {_path}')
    print(f'SAVE PATH : {config.output_dir}')
    train(config)
    
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ========== Train Finished ==========')

[2024-11-19 09:26:31.816560] ========== Train Start ==========
DEVICE : -1
MODEL NAME : gogamza/kobart-base-v2
SAVE PATH : ./result/corraction
[2024-11-19 09:26:31.816613] ====== Model Load Start ======


/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatib

[2024-11-19 09:26:33.710160] ====== Model Load Finished ======
[2024-11-19 09:26:33.710160] ====== Data Load Start ======
Train data loaded: 14400 entries
Validation data loaded: 1800 entries
[2024-11-19 09:26:34.001748] ====== Data Load Finished ======
[2024-11-19 09:26:34.001748] ====== Data Preprocessing Start ======


Map:   0%|          | 0/14400 [00:00<?, ? examples/s]/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1800/1800 [00:00<00:00, 2119.02 examples/s]
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: cor_sentence, token_type_ids, err_sentence. If cor_sentence, token_type_ids, err_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/transformers/optimization.py:30

[2024-11-19 09:26:41.572704] ====== Data Preprocessing Finished ======


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,5.553300,3.186116
100,3.328600,1.839988
150,2.244500,1.486019
200,1.861800,1.531946
250,2.094900,1.269094
300,1.751300,1.232995
350,1.790900,1.261871
400,1.438300,1.255949
450,1.392600,1.256849
500,1.540900,1.309069


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: cor_sentence, token_type_ids, err_sentence. If cor_sentence, token_type_ids, err_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1800
  Batch size = 1
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to ./result/corraction/checkpoint-50
Configuration saved in ./result/corraction/checkpoint-50/config.json
Mode

KeyboardInterrupt: 